## 2. В таблице приведены данные о содержании иммуноглобулина Ig A в сыворотке крови у больных пяти возрастных групп:

| № | Содержание Ig A(%)                         |
|:-:|:-------------------------------------------|
| 1 | 83, 85                                     |
| 2 | 84, 85, 85, 86, 86, 87                     |
| 3 | 86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90 |
| 4 | 89, 90, 90, 91                             |
| 5 | 90, 92                                     |

a) Определить влияние возраста на содержание иммуноглобулина в крови с помощью регриссионного анализа.

b) Провести попарное сравнение средних в рамках регрессионной модели, с учётом множественности проверяемых гипотез.

In [1]:
import pandas as pd
import statsmodels.api as sm

data = pd.DataFrame({
    'Age_Group': [1,1, 2,2,2,2,2,2, 3,3,3,3,3,3,3,3,3,3,3, 4,4,4,4, 5,5],
    'Ig_A': [83,85, 84,85,85,86,86,87, 86,87,87,87,88,88,88,88,88,89,90, 89,90,90,91, 90,92]
})

# Создание индикаторных переменных
base = 3
dummies = pd.get_dummies(data['Age_Group'], prefix='Age', dtype=int).drop(columns=[f'Age_{base}'])

dummies_clear = dummies

for k in range(1, 5 + 1):
    # Регрессия на остальные переменные
    if (k != base):
        model = sm.OLS(dummies_clear[f'Age_{k}'], sm.add_constant(dummies_clear.drop(f'Age_{k}', axis=1))).fit()
        print(f'R² для Age_{k}: {model.rsquared:.3f}')
        if (model.rsquared > 0.7):
            print(f'Age_{k} можно выбросить')
            dummies_clear = dummies_clear.drop(f'Age_{k}', axis=1)

# Регрессия
X = sm.add_constant(dummies_clear)
model = sm.OLS(data['Ig_A'], X).fit()
print(model.params)

R² для Age_1: 0.080
R² для Age_2: 0.149
R² для Age_4: 0.127
R² для Age_5: 0.080
const    87.818182
Age_1    -3.818182
Age_2    -2.318182
Age_4     2.181818
Age_5     3.181818
dtype: float64


In [ ]:
import numpy as np
from scipy.stats import f, ttest_ind

group_data = {
    1:  [83, 85],
    2:  [84, 85, 85, 86, 86, 87],
    3:  [86, 87, 87, 87, 88, 88, 88, 88, 88, 89, 90],
    4:  [89, 90, 90, 91],
    5:  [90, 92]
}
s_2 = [len(group_data[i + 1]) * (np.std(group_data[i + 1])) ** 2 / (len(group_data[i + 1]) - 1) for i in range(len(group_data))]

pairs = [(i, j) for i in range(1, 6) for j in range(i + 1, 6)]

# Проверка на равенство дисперсий
for pair in pairs:
    n = len(group_data[pair[0]])
    m = len(group_data[pair[1]])
    s2x = n / (n - 1) * np.std(group_data[pair[0]]) ** 2
    s2y = m / (m - 1) * np.std(group_data[pair[1]]) ** 2
    delta = s2x / s2y
    p_value = f.sf(delta, n - 1, m - 1)
    if (p_value < 0.05):
        print(f'Отвергаем H₀, группы {pair[0]}, {pair[1]}')


hb_p_values = {}
for pair in pairs:
    delta, p_value = ttest_ind(group_data[pair[0]], group_data[pair[1]], equal_var=True)
    if (p_value < 0.05):
        hb_p_values[pair] = p_value
    else:
        print(f'Незначимое различие средних между группами {pair[0]}, {pair[1]}')

hb_p_values = dict(sorted(hb_p_values.items(), key=lambda item: item[1]))

m = len(hb_p_values)
for pair, p_value in hb_p_values.items():
    if (p_value < 0.05 / m):
        m-=1
        print(f'Значимое различие средних между группами {pair[0]}, {pair[1]}')
    else:
        print(f'Незначимое различие средних между группами {pair[0]}, {pair[1]}')


Незначимое различие средних между группами 1, 2
Незначимое различие средних между группами 4, 5
Значимое различие средних между группами 2, 4
Значимое различие средних между группами 2, 3
Значимое различие средних между группами 2, 5
Значимое различие средних между группами 1, 3
Значимое различие средних между группами 1, 4
Значимое различие средних между группами 3, 4
Значимое различие средних между группами 3, 5
Значимое различие средних между группами 1, 5


In [3]:
# from statsmodels.formula.api import ols
# from statsmodels.stats.multicomp import pairwise_tukeyhsd
# import matplotlib.pyplot as plt


# # b) Попарное сравнение средних (ANOVA + Tukey HSD)
# # ANOVA
# anova_model = ols('Ig_A ~ C(Age_Group)', data=data).fit()
# anova_table = sm.stats.anova_lm(anova_model, typ=2)
# print("\nANOVA таблица:")
# print(anova_table)

# # Tukey HSD
# tukey = pairwise_tukeyhsd(endog=data.Ig_A, groups=data.Age_Group, alpha=0.05)
# print("\nРезультаты теста Тьюки:")
# print(tukey)

# # Визуализация
# tukey.plot_simultaneous()
# plt.xlabel('Уровень Ig A (%)')
# plt.ylabel('Группа')
# plt.title('Попарные сравнения (Tukey HSD)')
# plt.show()